# Inversion of Time Domain Electromagnetic Sounding (TDEM) Data

In [ ]:
import sys
import os
sys.path.insert(0, os.getcwd())

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import matplotlib

import walktem

In this notebook you will use a forward model for the TDEM response of a layered half space to estimate the resistivity structure of the ground where a TDEM sounding was acquired. The first part of the notebook will walk you through inversion of a synthetic sounding, then in the second part you will set up an inversion of a real sounding from the forelands of Sit' Tlein (Malaspina Glacier) in southern Alaska. Your goal is to determine if there is ice in the subsurface where the sounding was acquired. <font color="red"> Questions for you to answer and tasks for you to perform will be in red. Please answer questions and perform tasks in new markdown and/or code cell(s) immediately below the cell containing the question/task unless otherwise noted.</font>

<center>
    <img width="50%" src="https://eol.jsc.nasa.gov/DatabaseImages/EFS/highres/STS028/STS028-97-81.JPG">
</center>

# Part 0 - Information about model

You'll be using a forward model built with the [empymod](https://empymod.emsig.xyz/en/stable/) library. It computes the TDEM data one would expect to record from a layered half space representation of the Earth. To specify the layered half space the model takes as input a list of the resistivities of each layer and another list giving the thicknesses of each layer. It also requires some information related to the particulars of the TDEM instrument that we seek to simulate data for, but those inputs are handled for you in this notebook.

You'll be manipulating the layer resistivities and thicknesses manually to test out different TDEM scenarios and programatically (with an optimization algorithm) to estimate the resistivity structure that led to an observed dataset.

# Part 1 - Synthetic example

To begin you will generate a synthetic dataset with the forward model, add some noise, and then try to recover the starting earth model through an inversion.

## Generate synthetic data

The next cell loads a real data file to grab some parameters necessary for the forward model. Note that this is not strictly necessary, but it is quicker to do than making them up.

In [ ]:
# Load TDEM info
data = pd.read_csv("./geo4_tdem.txt", header=1)

# RX observation times
t_obs = data["t"].to_numpy()

# TX waveform
waveform = pd.read_csv("./tdem_waveform.txt", header=1) 
waveform = {"t": waveform["t"].to_numpy(), "i": waveform["i"].to_numpy()}

# TX loop size
loopsize = 40.0 # meters

Next we make up some stratigraphy - in this case a three layered earth - and use a utiliy function you are provided to visualize the stratigraphic column. Our synthetic stratigraphy will be a more conductive layer sandwiched between two resistive layers. There are many scenarios where this type of electro-stratigraphy might come up, such as a perched aquifer or a sulfide mineral deposit.

The `hatches` and `colors` arguments to the `strat_plot` function can be any [matplotlib hatch](https://matplotlib.org/stable/gallery/shapes_and_collections/hatch_style_reference.html) and [matplotlib color](https://matplotlib.org/stable/gallery/color/named_colors.html). 

In [ ]:
layer_thicknesses = [100, 20] # Two thicknesses (bottom layer is assumed infinite)
layer_resistivities = [100, 5, 150] # Three resistivities
walktem.strat_plot(layer_thicknesses, layer_resistivities, hatches=["o", "/", "x"], colors=["brown", "grey", "sienna"])

Then we will use the forward model for the response of a TDEM instrument over a layered earth to generate a synthetic TDEM sounding.

The forward modeling function is `walktem.walktem` - note the first two arguments:

 - `layer_resistivities` - a list of the resistivities of each layer in the model (unit: ohm-m)
 - `layer_thicknesses` - a list of the thicknesses of each layer (unit: m)

The `layer_thicknesses` list is one item shorter than `layer_resistivities` because the bottom layer is assumed to be infinitely thick.

The final three arguments pass instrument-related information.

In [ ]:
# Forward model
synth = walktem.walktem(layer_resistivities, layer_thicknesses, t_obs, waveform, loopsize)

# Make plot
plt.loglog(t_obs, synth, "k.-")
plt.xlabel("Time (s)")
plt.ylabel("TDEM Receive Loop Voltage (V)")
plt.show()

The forward model returns the expected receive loop voltage recorded by the instrument at each observation time. <font color="red">What electromagnetic quantity is the receive loop voltage voltage porportional to?</font>

<font color="red">In a new code cell below, run and plot forward models for 0.1 ohm-m and 100 ohm-m homogenous half spaces then plot them together. In a sentence or two comment on the differences between the two.</font>

Hint... for a 500 ohm-m homogenous half space
```
layer_thicknesses = []
layer_resistivities = [500.0]
```

<font color="red">Try out the following two scenarios and plot them together. Are the predicted data significantly different? What does this tell you about the limitations of the TDEM method?</font>

Scenario 1
```
layer_thicknesses = [50.0, 0.1]
layer_resistivities = [100, 0.1, 100.0]
```

Scenario 2
```
layer_thicknesses = [50.0, 10.0]
layer_resistivities = [100.0, 10.0, 100.0]
```

## Invert synthetic data
The forward model is useful to manually experiment with different resistivity structure scenarios, but it can also be used to estimate the resistivity structure of the Earth given some observed data by finding a resistivity structure that yields a good fit to the observed data. This process is called inversion.

To start we need to define what a good fit to the observed data means. We will choose the L2 norm, $||data-model||^2_2$, scaled by the data value. The function `misfit_nlyr` implements this. It would probably also be good to include something about the uncertainty of each data point, but we'll ignore that in this notebook.


Next, we will need to pick a starting earth model for the inversion, `x0`. This is a very important input - just as important as the choice of misfit function. Choosing a bad initial `x0` can make it impossible for your optimizer to find a good-fitting solution. For this synthetic example we know the correct solution, but with real data you never do and must often use other knowledge to come up with a good `x0` I'll choose something that generally represents the known electro-stratigraphy (resistive, conductive, resistive) but is notably different from what we know to be the true model.

In [ ]:
## Generate synthetic "observed" data
true_layer_resistivities = [100.0, 5.0, 150.0]
true_layer_thicknesses = [100.0, 20.0]

# Forward model
synth = walktem.walktem(true_layer_resistivities, true_layer_thicknesses, t_obs, waveform, loopsize)

# Add a tiny bit of noise
synth += np.random.normal(loc=0, scale=.01*synth, size=len(synth))

In [ ]:
# Define misfit function
def misfit_nlyr(x, data, t_obs, waveform, loopsize):
    # Split x into resistivity and thickness
    res, thk = walktem.split_x(x)
    model = walktem.walktem(res, thk, t_obs, waveform, loopsize)
    return np.sum(np.power(data-model, 2)/((.1*data)**2))

# Define starting earth model for the inversion
init_layer_resistivities = [80.0, 1.0, 110.0]
init_layer_thicknesses = [75.0, 10.0]

x0 = init_layer_resistivities + init_layer_thicknesses

# We also need to provide parameter bounds for the inversion, here I am constraining the resistivity to between 1 and 500 ohm meters
# and the layer thickness to between 0.5 and 200 meters
bounds = [(1, 500)]*3 + [(0.5, 200)]*2

# Forward model
x0_model = walktem.walktem(x0[:(len(x0)+1)//2], x0[(len(x0)+1)//2:], t_obs, waveform, loopsize)

# Make plot
plt.loglog(t_obs, synth, "k.-", label="Synthetic Data")
plt.loglog(t_obs, x0_model, "r.-", label="Initial Guess")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("TDEM Receive Loop Voltage (V)")
plt.show()

The initial guess somewhat replicates the shape of the observed decay curve, but is note the same. Now we can run a minimization algorithm to seek a resistivity structure that better matches the observed data. The function call below implements this. See [this page](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) for information about all of the arguments. Optimization (a.k.a. minimization) is a complex topic, worthy of class(es) in its own right. The next cell will take several of seconds to run.

In [ ]:
# Run optimizer
res = scipy.optimize.minimize(
    misfit_nlyr,
    x0,
    method="nelder-mead",
    args=(synth, t_obs, waveform, loopsize),
    bounds=bounds,
    options={"xatol": 1e-3, "maxfev": len(x0) * 300},
)

print(res)

You should see `message: Optimization terminated successfully.` at the top of the ouput from the cell above if everything worked well so far. Let's plot the result.

In [ ]:
xinv = res.x
inv_layer_resistivities, inv_layer_thicknesses = walktem.split_x(xinv)
inv = walktem.walktem(inv_layer_resistivities, inv_layer_thicknesses, t_obs, waveform, loopsize)

# Make plot
plt.loglog(t_obs, synth, "k.-", label="Observed (synthetic) daya")
plt.loglog(t_obs, inv, "r-", label="Inverted model")
plt.legend(frameon=False)
plt.xlabel("Time (s)")
plt.ylabel("TDEM Receive Loop Voltage (V)")

# Resistivity column
walktem.strat_plot(inv_layer_thicknesses.tolist(), inv_layer_resistivities.tolist(), hatches=["o", "/", "o"], colors=["brown", "grey", "brown"], labels=["300 ohm-m", "1 ohm-m"])

Great! We've recovered something close to the known original model. Note the values are not exactly the same - we added some noise to the synthetic data so would not expect them to be. Now it is your turn to invert real data.

# Part 2 - Real data

The code snippet below loads a TDEM data file acquired in the forelands of Sit' Tlein. Do some research on the expected resistivities of materials you might find in the forelands of a glacier (e.g., glacial till) and of ice. Your task will be to invert for a three layer model that reasonably explains the observed data and decide whether there is potentially ice in the subsurface at the sounding site.

Note - your inverted model should be reasonable but there are not exact resistivity/thickness values we are seeking. Your result will vary based on your initial model.

Please limit your solutions to resistivities between 1 and 100,000 ohm-m and thicknesses between 0.5 and 200 m. So your `bounds` variable should be
```
bounds = [(1, 1e5)]*3 + [(0.5, 200)]*2
```
The bounds variable is provided, just don't change it. You are given a similar structure to the synthetic example above to work in

### Load and plot the observed data

In [ ]:
# Load TDEM data
data = pd.read_csv("./geo4_tdem.txt", header=1)
t_obs = data["t"].to_numpy()
v_obs = data["v"].to_numpy()

waveform = pd.read_csv("./tdem_waveform.txt", header=1)
waveform = {"t": waveform["t"].to_numpy(), "i": waveform["i"].to_numpy()}
loopsize = 40.0

# Plot
plt.loglog(t_obs, v_obs, "k.-")
plt.xlabel("Time (s)")
plt.ylabel("TDEM Receive Loop Voltage (V)")
plt.show()

### Find a starting model

<font color="red">Fill in the `layer_resistivities` and `layer_thicknesses` variable in the cell below to define a starting model to begin the inversion.</font> Remember that the starting model is an important input to the inversion and the predicted data from it should be not very dissimilar to the observed data in order to achieve a good fit during the inversion.

You are at a disadvantage because you likely have not worked in the forelands of Sit` Tlein before, so you have not developed intuition about the materials that are present. If you had visited the site you would have noted the following facts:
- There was significant surficial evidence of widespread shallow subsurface ice (e.g., thermokarst ponds).
- Much of Sit` Tlein's forelands are made up of glacial till and fluvially reworked glacial till.
- There is a lot of water, and the ground is damp or saturated in most places.

In [ ]:
# Finding a starting model
layer_resistivities = [RES_1, RES_2, RES_3]
layer_thicknesses = [THICK_1, THICK_2]

# Forward model
synth = walktem.walktem(layer_resistivities, layer_thicknesses, t_obs, waveform, loopsize)

# Make plot
plt.loglog(t_obs, synth, "r.-")
plt.xlabel("Time (s)")
plt.ylabel("TDEM Receive Loop Voltage (V)")

plt.loglog(t_obs, v_obs, "k.-")
plt.show()

### Run the optimizer

Note - depending on your starting model you may not get the same `message: Optimization terminated successfully.` status as was output for the synthetic example. If you still get a reasonably well fitting inversion you don't need to worry about it. If the predicted TDEM sounding from your inverted resistivity structure does not fit well then you need to change your starting model.

In [ ]:
# Invert for an earth model
x0 = layer_resistivities + layer_thicknesses

# We also need to provide parameter bounds for the inversion, here I am constraining the resistivity to between 1 and 500 ohm meters
# and the layer thickness to between 0.5 and 200 meters
bounds = [(1, 1e5)]*3 + [(0.5, 200)]*2

# Forward model
#x0_model = walktem.walktem(x0[:(len(x0)+1)//2], x0[(len(x0)+1)//2:], t_obs, waveform, loopsize)

# Run optimizer
res = scipy.optimize.minimize(
    misfit_nlyr,
    x0,
    method="nelder-mead",
    args=(v_obs, t_obs, waveform, loopsize),
    bounds=bounds,
    options={"xatol": 1e-3, "maxfev": len(x0) * 300},
)

print(res)

### Plot your inversion and the observed data

In [ ]:
xinv = res.x
inv_layer_resistivities, inv_layer_thicknesses = walktem.split_x(xinv)
inv = walktem.walktem(inv_layer_resistivities, inv_layer_thicknesses, t_obs, waveform, loopsize)

# Make plot
plt.loglog(t_obs, v_obs, "k.-")
plt.loglog(t_obs, inv, "r-")
plt.xlabel("Time (s)")
plt.ylabel("TDEM Receive Loop Voltage (V)")

# Resistivity column
walktem.strat_plot(inv_layer_thicknesses.tolist(), inv_layer_resistivities.tolist(), hatches=["o", "/", "x"], colors=["brown", "grey", "sienna"], labels=["300 ohm-m", "1 ohm-m"])

<font color="red">What does the inverted resistivity structure tell you about the composiiton of the subsurface at the sounding site? Is there ice? If so, how thick is the ice layer? Why might a thin non-ice layer above ice not be resolved by the TDEM method?</font> 

To sumbit this lab download the notebook file (`File > Download`) and submit it on Canvas.